In [1]:
from  piCalculation import *
from menschGame import *


In [32]:
num_points = 100000
square_side = 2
points = generate_points(num_points, square_side)
pi_estimate = estimate_pi(points, square_side)
print(f"Estimated value of Pi: {pi_estimate}")
print(f"Accuracy: {round(100 - abs(math.pi - pi_estimate) / math.pi * 100, 2)}%")

Estimated value of Pi: 3.13176
Accuracy: 99.69%


In [34]:
test = Mensch_Simulation(100000)
test.simulate()

Counter({'Player 4': 26743,
         'Player 3': 24951,
         'Player 1': 24557,
         'Player 2': 23749})

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, expon, uniform

num_samples = 1000

sample_sizes = [10, 100, 1000]

def generate_sample_means(distribution, sample_size):
    sample_means = []
    for _ in range(num_samples):
        sample = distribution.rvs(size=sample_size)
        sample_means.append(np.mean(sample))
    return sample_means

def plot_sample_means(sample_means, sample_size, distribution):
    plt.figure(figsize=(10, 6))
    plt.hist(sample_means, bins=30, density=True, alpha=0.6, color='g', label='Sample Means Histogram')
    
    mu = np.mean(sample_means)
    sigma = np.std(sample_means)
    
    x = np.linspace(min(sample_means), max(sample_means), 100)
    plt.plot(x, norm.pdf(x, mu, sigma), 'r-', label='Expected Normal Distribution')
    
    plt.title(f'Sample Size: {sample_size}, Distribution: {distribution.name}')
    plt.xlabel('Sample Means')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

distributions = [norm, expon, uniform]
for distribution in distributions:
    print(f"Analyzing {distribution.name} distribution...")
    for sample_size in sample_sizes:
        print(f"\tSample Size: {sample_size}")
        sample_means = generate_sample_means(distribution, sample_size)
        plot_sample_means(sample_means, sample_size, distribution)
